In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
import csv

import sys

sys.path.append("../../")

In [ ]:
show_animation = True
WORLD_SIZE = 500
# Vehicle parameters
NX = 5  # x = x,y,v,yaw,omega
NU = 2  # a = [velocity,steer]
DT = 0.5

# Vehicle parameters描画用らしい
LENGTH = 4.5  # [m]
WIDTH = 2  # [m]
BACKTOWHEEL = 1  # [m]
WHEEL_LEN = 0.3  # [m]
WHEEL_WIDTH = 0.2  # [m]
TREAD = 0.7  # [m]
WB = 2.5  # [m]

MAX_STEERING_ANGLE = math.radians(45.0)  # maximum steering angle [rad]
MAX_DSTEER = math.radians(30.0)  # maximum steering speed [rad/s]

In [ ]:
class Config():
    # simulation parameters

    def __init__(self):
        # robot parameter
        self.max_speed = 5  # [m/s]
        self.min_speed = 0  # [m/s]
        self.max_yawrate = 40.0 * math.pi / 180.0  # [rad/s]
        self.max_accel = 0.5  # [m/ss]
        self.max_dyawrate = 40.0 * math.pi / 180.0  # [rad/ss]
        self.v_reso = 0.1  # [m/s]
        self.yawrate_reso = 1 * math.pi / 180.0  # [rad/s]
        self.dt = 1.0  # [s]
        self.predict_time = 1.0  # [s]
        self.to_goal_cost_gain = 0.0
        self.speed_cost_gain = 1.0
        self.ob_cost_gain = 0.0
        self.to_goal_dis_cost_gain = 1.0
        self.robot_radius = 3.0  # [m]

In [ ]:
def motion(x, u, dt):
    # motion model for two-wheel robot x,y,orient,v,steering

    x[2] += u[1] * dt
    x[0] += u[0] * math.cos(x[2]) * dt
    x[1] += u[0] * math.sin(x[2]) * dt
    x[3] = u[0]
    x[4] = u[1]

    return x

In [ ]:
def calc_dynamic_window(x, config):
    # Dynamic window from robot specification
    Vs = [config.min_speed, config.max_speed,
          -config.max_yawrate, config.max_yawrate]

    # Dynamic window from motion model
    Vd = [x[3] - config.max_accel * config.dt,
          x[3] + config.max_accel * config.dt,
          x[4] - config.max_dyawrate * config.dt,
          x[4] + config.max_dyawrate * config.dt]

    #  [vmin,vmax, yawrate min, yawrate max]
    dw = [max(Vs[0], Vd[0]), min(Vs[1], Vd[1]),
          max(Vs[2], Vd[2]), min(Vs[3], Vd[3])]

    return dw

In [ ]:
def calc_trajectory(xinit, v, y, config):
    x = np.array(xinit)
    traj = np.array(x)
    time = 0
    while time <= config.predict_time:
        x = motion(x, [v, y], config.dt)
        traj = np.vstack((traj, x))
        time += config.dt

    return traj

In [ ]:
def calc_final_input(x, u, dw, config, goal, ob):
    xinit = x[:]
    min_cost = 10000.0
    min_u = u
    min_u[0] = 0.0
    best_traj = np.array([x])

    # evalucate all trajectory with sampled input in dynamic window
    for v in np.arange(dw[0], dw[1], config.v_reso):
        for y in np.arange(dw[2], dw[3], config.yawrate_reso):
            traj = calc_trajectory(xinit, v, y, config)

            # calc cost
            to_goal_cost = config.to_goal_cost_gain * calc_to_goal_cost(traj, goal, config)
            to_goal_dis_cost = config.to_goal_dis_cost_gain * calc_to_goal_dis_cost(traj, goal, config)
            speed_cost = config.speed_cost_gain * \
                         (config.max_speed - traj[-1, 3])  # traj[-1,3] velocity
#             ob_cost = config.ob_cost_gain * calc_obstacle_cost(traj, ob, config)
            # print(ob_cost)

            final_cost = to_goal_cost + speed_cost + to_goal_dis_cost

            # search minimum trajectory
            if min_cost >= final_cost:
                min_cost = final_cost
                min_u = [v, y]
                best_traj = traj

    return min_u, best_traj

In [ ]:
def calc_whole_subgoal_final_input(x, u, dw, config, goal_loc, main_index):
    xinit = x[:]
    min_cost = 10000.0
    min_u = u
    min_u[0] = 0.0
    best_traj = np.array([x])

    # evalucate all trajectory with sampled input in dynamic window
    for v in np.arange(dw[0], dw[1], config.v_reso):
        for y in np.arange(dw[2], dw[3], config.yawrate_reso):
            traj = calc_trajectory(xinit, v, y, config)

            # calc cost
            # to_goal_cost = config.to_goal_cost_gain * calc_to_goal_cost(traj, config)
            to_goal_dis_cost = config.to_goal_dis_cost_gain * calc_whole_sub_goal_dis_cost(traj, goal_loc, config,
                                                                                           main_index)

            speed_cost = config.speed_cost_gain * \
                         (config.max_speed - traj[-1, 3])  # traj[-1,3] velocity
#             ob_cost = config.ob_cost_gain * calc_obstacle_cost(traj, ob, config)
            # print(ob_cost)

            final_cost = to_goal_dis_cost + speed_cost

            # search minimum trajectory
            if min_cost >= final_cost:
                min_cost = final_cost
                min_u = [v, y]
                best_traj = traj

    return min_u, best_traj

In [ ]:
def calc_obstacle_cost(traj, ob, config):
    # calc obstacle cost inf: collistion, 0:free

    skip_n = 2
    minr = float("inf")

    for ii in range(0, len(traj[:, 1]), skip_n):
        for i in range(len(ob[:, 0])):
            ox = ob[i, 0]
            oy = ob[i, 1]
            dx = traj[ii, 0] - ox
            dy = traj[ii, 1] - oy

            r = math.sqrt(dx ** 2 + dy ** 2)
            if r <= config.robot_radius:
                return float("Inf")  # collision

            if minr >= r:
                minr = r

    return 1.0 / minr  # OK

In [ ]:
def calc_to_goal_cost(traj, goal, config):
    # calc to goal cost. It is 2D norm.

    goal_magnitude = math.sqrt(goal[0] ** 2 + goal[1] ** 2)
    traj_magnitude = math.sqrt(traj[-1, 0] ** 2 + traj[-1, 1] ** 2)
    dot_product = (goal[0] * traj[-1, 0]) + (goal[1] * traj[-1, 1])
    error = dot_product / (goal_magnitude * traj_magnitude)
    error_angle = math.acos(error)
    cost = config.to_goal_cost_gain * error_angle

    return cost

In [ ]:
def calc_to_goal_dis_cost(traj, goal, config):
    # calculate to goal cost by distance

    distance = np.sqrt((goal[0] - traj[-1, 0]) ** 2 + (goal[1] - traj[-1, 1]) ** 2)
    to_goal_dis_cost = config.to_goal_dis_cost_gain * distance

    return to_goal_dis_cost

In [ ]:
def calc_whole_sub_goal_dis_cost(traj, goal_loc, config, main_index):
    # calculate whole sub goal cost by distance

    weight_1 = 0.6
    weight_2 = 0.4
    # the initail index of the sub goal
    sub_goal_distance = weight_1 * np.sqrt(
        (goal_loc[main_index][0] - traj[-1, 0]) ** 2 + (goal_loc[main_index][1] - traj[-1, 1]) ** 2) + \
                        weight_2 * np.sqrt(
        (goal_loc[main_index + 1][0] - traj[-1, 0]) ** 2 + (goal_loc[main_index + 1][1] - traj[-1, 1]) ** 2)

    sub_goal_cost = config.to_goal_cost_gain * sub_goal_distance
    return sub_goal_cost

In [ ]:
def dwa_control(x, u, config, goal, ob):
    # Dynamic Window control

    dw = calc_dynamic_window(x, config)

    u, traj = calc_final_input(x, u, dw, config, goal, ob)

    return u, traj


def dwa_control_all(x, u, config, goal_loc, main_index):
    # Dynamic Window control with all sub goal

    dw = calc_dynamic_window(x, config)

    u, traj = calc_whole_subgoal_final_input(x, u, dw, config, goal_loc, main_index)

    return u, traj

In [ ]:
def plot_arrow(x, y, yaw, length=0.5, width=0.1):  # pragma: no cover
    plt.arrow(x, y, length * math.cos(yaw), length * math.sin(yaw),
              head_length=width, head_width=width)
    plt.plot(x, y)

In [ ]:
def goal_arrived(x, u, config, goal):
    arrive = False
    if math.sqrt((x[0] - goal[0]) ** 2 + (x[1] - goal[1]) ** 2) <= config.robot_radius:
        print("Goal!!")
        arrive = True
        return arrive

In [ ]:
def plot_car(x, y, yaw, steer=0.0, cabcolor="-r", carcolor="-m"):
    outline = np.matrix([[-BACKTOWHEEL, (LENGTH - BACKTOWHEEL), (LENGTH - BACKTOWHEEL), -BACKTOWHEEL, -BACKTOWHEEL],
                         [WIDTH / 2, WIDTH / 2, - WIDTH / 2, -WIDTH / 2, WIDTH / 2]])

    fr_wheel = np.matrix([[WHEEL_LEN, -WHEEL_LEN, -WHEEL_LEN, WHEEL_LEN, WHEEL_LEN],
                          [-WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD,
                           -WHEEL_WIDTH - TREAD]])

    rr_wheel = np.copy(fr_wheel)

    fl_wheel = np.copy(fr_wheel)
    fl_wheel[1, :] *= -1
    rl_wheel = np.copy(rr_wheel)
    rl_wheel[1, :] *= -1

    Rot1 = np.matrix([[math.cos(yaw), math.sin(yaw)],
                      [-math.sin(yaw), math.cos(yaw)]])
    Rot2 = np.matrix([[math.cos(steer), math.sin(steer)],
                      [-math.sin(steer), math.cos(steer)]])

    fr_wheel = (fr_wheel.T * Rot2).T
    fl_wheel = (fl_wheel.T * Rot2).T
    fr_wheel[0, :] += WB
    fl_wheel[0, :] += WB

    fr_wheel = (fr_wheel.T * Rot1).T
    fl_wheel = (fl_wheel.T * Rot1).T

    outline = (outline.T * Rot1).T
    rr_wheel = (rr_wheel.T * Rot1).T
    rl_wheel = (rl_wheel.T * Rot1).T

    outline[0, :] += x
    outline[1, :] += y
    fr_wheel[0, :] += x
    fr_wheel[1, :] += y
    rr_wheel[0, :] += x
    rr_wheel[1, :] += y
    fl_wheel[0, :] += x
    fl_wheel[1, :] += y
    rl_wheel[0, :] += x
    rl_wheel[1, :] += y

    plt.plot(np.array(outline[0, :]).flatten(), np.array(outline[1, :]).flatten(), carcolor)
    plt.plot(np.array(fr_wheel[0, :]).flatten(), np.array(fr_wheel[1, :]).flatten(), carcolor)
    plt.plot(np.array(rr_wheel[0, :]).flatten(), np.array(rr_wheel[1, :]).flatten(), carcolor)
    plt.plot(np.array(fl_wheel[0, :]).flatten(), np.array(fl_wheel[1, :]).flatten(), carcolor)
    plt.plot(np.array(rl_wheel[0, :]).flatten(), np.array(rl_wheel[1, :]).flatten(), carcolor)
    plt.plot(x, y, "x")

In [ ]:
def land_generator(low,high,size):
    sample_x=np.random.uniform(low, high, size)
    sample_y=np.random.uniform(low, high, size)
    axis=np.stack([sample_x,sample_y],1)
    return axis

def get_ot(st,lmap,max_range):#[dis,rad_sin,rad_cos]
    dis = np.sqrt((st[0]-lmap[0])**2+(st[1]-lmap[1])**2)
    angle = np.arctan2((lmap[1]-st[1]),(lmap[0]-st[0]))-st[2]
    if (dis < max_range[0]) and (np.abs(angle) < max_range[1]):
        return [dis,np.sin(angle),np.cos(angle)]
    return [0.0,0.0,0.0]

def get_all_ot(st,lmap,max_range):
    measure = []
    for l in range(len(lmap)):
        measure.append( get_ot(st,lmap[l],max_range) )
    return measure

In [ ]:
def main_run(goals,goal_loc,ob,landmark,data_num):
    #初期化
    # initial state [x(m), y(m), yaw(rad), v(m/s), omega(rad/s)]
    x = np.array([2.0, 0.0, 0.0, 0.0, 0.0])
    u = np.array([0.0, 0.0])
    config = Config()
    traj = np.array(x)
    index = 0  # the index of the subgoal
    exit_simu = False
    
    timestep = 1 #時刻step保持
    lf = open('landmark_data.csv','w')
    writer = csv.writer(lf,lineterminator='\n')
    writer.writerows(landmark)
    lf.close()
    
    max_range = [1000,1000] #計測距離
    
    file_name="random_data/vehicle_motion_data"+str(data_num)+".csv"
    with open(file_name,'w') as f:
        while not exit_simu:
            #csv書き込み
            ot = get_all_ot(x,landmark,max_range)#[dis,rad_sin,rad_cos]
            print(np.shape(ot))
            ot = np.reshape(ot,(1,len(ot)*3)) #一列に
            data=[[ timestep,x[0],x[1],np.sin(x[2]),np.cos(x[2]),u[0],np.sin(u[1]),np.cos(u[1])]]
            data=np.concatenate((data,ot),1) #[time,s_x,s_y,s_yaw,uv,ur,ot[0],,,,ot[N]]
            print(np.shape(data))
            writer = csv.writer(f, lineterminator='\n')
            writer.writerows(data)
            
            u, ltraj = dwa_control(x, u, config, goals[index], ob)

            x = motion(x, u, config.dt)
            timestep +=1
            print(u)
            print(x)
            traj = np.vstack((traj, x))  # store state history

            if show_animation:
                plt.cla()
                plt.plot(ltraj[:, 0], ltraj[:, 1], "-g")
                plt.plot(x[0], x[1], "xr")
                plt.plot(goal_loc[0][0], goal_loc[0][1], "go")
                plt.plot(goal_loc[1][0], goal_loc[1][1], "go")
                plot_car(x[0], x[1], x[2], u[1] * 0.1)
                plt.plot(goal_loc[2][0], goal_loc[2][1], "go")
                plt.plot(goal_loc[3][0], goal_loc[3][1], "go")
#                 for l in range(len(landmark)):
#                     plt.plot(landmark[l][0],landmark[l][1],"co")

                plot_arrow(x[0], x[1], x[2])
                plt.axis("equal")
                plt.grid(True)
                plt.pause(0.0001)

            if goal_arrived(x, u, config, goals[index]) and index <= len(goal_loc) - 1:
                index = index + 1
                if index == len(goals):
                    #最終書き込み
                    ot = get_all_ot(x,landmark,max_range)
                    ot = np.reshape(ot,(1,len(ot)*3)) #一列に
                    data=[[ timestep,x[0],x[1],np.sin(x[2]),np.cos(x[2]),u[0],np.sin(u[1]),np.cos(u[1]) ]]
                    data=np.concatenate((data,ot),1)
                    writer = csv.writer(f, lineterminator='\n')
                    writer.writerows(data)
                    exit_simu = True
                    break
                print(index)

    print("Done")
    if show_animation:
        plt.plot(traj[:, 0], traj[:, 1], "-m")
        plt.pause(0.0001)

    plt.show()

In [ ]:
def main():
    np.random.seed(4)
    #print(__file__ + " start!!")
    
    # goal position [x(m), y(m)]
    goal_loc = np.array([
        [2, 0],
        [50, 0],
        [90, 90],
        [90, -90],
    ])
    # goal_loc = np.array([[10, 10],
    #                   [2, 8],
    #                    [6, 6],
    #                    [10, 8]])

    # print(len(goal_loc))
    # obstacles [x(m) y(m), ....]
    ob = np.array([[75, -50],
                   [75, 50],
                   ])
    
    landmark = land_generator(0.0,100,10) #ランドマーク生成
    
    for i in range(500):
        first_point = np.random.randint(1,4,1)
        goals = np.array(goal_loc[first_point])
        while True:
            second_point = np.random.randint(0,4,1)
            if second_point != first_point:
                break
        goals=np.append(goals,np.array(goal_loc[second_point]),axis=0)
#         np.concatenate([goals,goal_loc[second_point]],dim=2)
        
        main_run(goals,goal_loc,ob,landmark,i)
    
    

In [ ]:

if __name__ == '__main__':
    main()
